In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import sys
import os

# modify the path below accordingly
PATH = "/content/drive/MyDrive/compvision/gesture_face_features"

sys.path.append(os.path.join(PATH, "code"))

In [8]:
# !pip install mediapipe

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# importing .py files from /code
from utils import *
from config import *
from heatmap import Heatmap
from tsne import test_tsne
# from hand_extraction import extract_hand_mp

In [11]:
print(available_models)
print(available_letters)

['VGG19', 'ResNet50', 'InceptionV3', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB6', 'squeezenet']
['A', 'H', 'L', 'N', 'O', 'P', 'R']


In [24]:
# Configurations
img_type="sign"  # it loads both sign and face in any case (fix the function)
letter = available_letters[0]
is_random = False
load_data = True
img_numbers = participants[0]
model_name = available_models[2]

In [25]:
print(f"Configuring {model_name} for {img_type} images for letter {letter}")
conf = Config(PATH,
                img_type=img_type,
                letter=letter,
                model_name=model_name,
                load_data=load_data,
                is_random=is_random,
                img_numbers=img_numbers,
              )

Configuring InceptionV3 for sign images for letter A
Loading data...
Data loaded successfully.
Setting up model InceptionV3...
Model was set up successfully.


In [26]:
img_numbers

[2914,
 2915,
 2916,
 2917,
 2918,
 2919,
 2920,
 2921,
 2922,
 2923,
 2924,
 2925,
 2926,
 2927,
 2928,
 2929,
 2930,
 2931,
 2932,
 2933,
 2934,
 2935,
 2936,
 2937,
 2938,
 2939,
 2940,
 2941,
 2942,
 2943,
 2944,
 2945,
 2946,
 2947,
 2948,
 2949,
 2950,
 2871,
 2872,
 2873,
 2874,
 2875,
 2876,
 2877,
 2878,
 2879,
 2880,
 2881,
 2882,
 2883,
 2884,
 2885,
 2886,
 2887,
 2888,
 2889,
 2890,
 2891,
 2892,
 2893,
 2894,
 2895,
 2896,
 2897,
 2898,
 2899,
 2900,
 2901,
 2902,
 2903]

In [17]:
# hm = Heatmap(*conf.heatmap_config)
# hm.generate_heatmaps_row(imgs_per_row=6)

In [18]:
test_tsne(conf, len(letter))

ValueError: need at least one array to concatenate